<a href="https://colab.research.google.com/github/phanng0605/PytorchTransformer/blob/main/Transformer_Pytorch_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Made by: Phan Nguyen
### This is a note book I made to implement the Transformer model from scratch with the Pytorch framework
<b>Link to the paper (Attention is all you need): https://arxiv.org/pdf/1706.03762.pdf</b>

## 0. Import and install Pytorch

In [ ]:
%pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn

## 1. Self Attention

In [ ]:
class SelfAttention(nn.Module):
    # this function is to set the parameters that are about to be passed into the forward function
    def __init__(self, embed_size, heads):
       # create an instance of this class
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert(self.head_dim * heads == embed_size), "Embedding size must be divisibled by number of heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)

        self.fc_out = nn.Linear(heads*self.head_dim, embed_size)

    def forward(self, values, keys, queries, mask):
        # get number of training examples
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        # split embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # matrix multiplication
        energy = torch.einsum("nqhd, nkhd -> nhqk",  [queries, keys])

        #queries shape = (N, query_len, heads, head_dim)
        #keys shape = (N, key_len, heads, head_dim)
        #energy shape = (N, heads, query_len, key_len)

        if mask is not None:
              energy = energy.masked_fill(mask == 0, float(-1e20))

        #compute attention
        attention = torch.softmax(energy / self.embed_size ** (1/2), dim = 3)

        #output
        out = torch.einsum("nhql, nlhd -> nqhd", [attention, values]).reshape(N, query_len, self.heads*self.head_dim)
        #attention shape = (N, heads, query_len, key_len)
        #values shape = (N, value_len, heads, head_dim)
        #after einsum, output shape = (N, query_len, heads, head_dim), then we flatten its last two dimension (heads*self.head_dim)

        # pass through the fully connected layer
        out = self.fc_out(out)
        return out

## 2. Transformer Block
### (To feed in the Encoder)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)

        #normalization layer
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        #feed forward layer include resizing the input size & apply reLU
        self.feed_forward = nn.Sequential(
              nn.Linear(embed_size, forward_expansion * embed_size),
              nn.ReLU(),
              nn.Linear(forward_expansion * embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        #call the forward method (since the SelfAttention class has only 1 method)
        attention = self.attention(value, key, query, mask)

        #Add & Normalize (including skip connection)
        x = self.dropout(self.norm1(attention + query))

        forward = self.feed_forward(x)

        out = self.dropout(self.norm2(x + forward))

        return out


## 3. Encoder Implementation

In [ ]:
class Encoder(nn.Module):
        def __init__(
            self,
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length):

            super(Encoder, self).__init__()
            self.embed_size = embed_size
            self.device = device
            self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
            self.positional_embedding = nn.Embedding(max_length, embed_size)

            self.layers = nn.ModuleList(
                  [
                        TransformerBlock(
                              embed_size,
                              heads,
                              dropout = dropout,
                              forward_expansion = forward_expansion
                        )
                        for _ in range(num_layers)
                  ]
            )
            self.dropout = nn.Dropout(dropout)

        def forward(self, x, mask):
            N, seq_length = x.shape
            positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

            out = self.dropout(self.word_embedding(x) + self.positional_embedding(positions))

            for layer in self.layers:
                  out = layer(out, out, out, mask)

            return out

## 4. Decoder Block

In [ ]:
# DECODER BLOCK
class DecoderBlock(nn.Module):
    def __init__(self,
                 embed_size,
                 heads,
                 forward_expansion,
                 dropout,
                 device):

        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        # masked self attention
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))

        # passing this query, value, key to a whole transformer block
        # forward(self, value, key, query, mask):
        out = self.transformer_block(value, key, query, src_mask)
        return out

## 5. Decoder Implementation


In [ ]:
# DECODER IMPLEMENTATION
class Decoder(nn.Module):
    def __init__(self,
                 trg_vocab_size,
                 embed_size,
                 num_layers,
                 heads,
                 forward_expansion,
                 dropout,
                 device,
                 max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(
                    embed_size,
                    heads,
                    forward_expansion=forward_expansion,
                    dropout=dropout,
                    device=device
                ) for _ in range(num_layers)
            ]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(
            N, seq_length).to(self.device)

        # add positional encoding
        x = self.dropout(self.word_embedding(
            x) + self.positional_embedding(positions))

        # N decoder blocks
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out

## 6. Transformer Combination

In [ ]:
# COMBINE TO GET FULL TRANSFORMER
class Transformer(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 trg_vocab_size,
                 src_pad_idx,
                 trg_pad_idx,
                 embed_size=256,
                 num_layers=6,
                 forward_expansion=4,
                 heads=8,
                 dropout=0,
                 device='cuda',
                 max_length=100):
        super(Transformer, self).__init__()
        # call Encoder
        self.device = device
        self.encoder = Encoder(src_vocab_size,
                               embed_size,
                               num_layers,
                               heads,
                               device,
                               forward_expansion,
                               dropout,
                               max_length)

        # call Decoder
        self.decoder = Decoder(trg_vocab_size,
                               embed_size,
                               num_layers,
                               heads,
                               forward_expansion,
                               dropout,
                               device,
                               max_length)

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))
                              ).expand(N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

## 7. Main (To test the implementation)

In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    )
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )
    out = model(x, trg[:, :-1])
    print(out.shape)

cuda
torch.Size([2, 7, 10])
